<a href="https://colab.research.google.com/github/avaneeshp23/GenAIhackathon/blob/hackathon/groqcloud_deepseek_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.7 MB/s eta 0:00:00


In [1]:
import os

from getpass import getpass
GROQ_API_KEY = getpass('Please enter your GROQ API Token here: ')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

Please enter your GROQ API Token here: ··········


In [3]:
from groq import Groq
import pandas as pd
import json


In [4]:
client = Groq(api_key=os.environ['GROQ_API_KEY'])

In [ ]:
# def classify_ticket(text):
#     prompt = f"""
#     Classify the following IT ticket text according to these instructions:

#     **Department:** Choose from: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry.
#     **Priority:** Determine the urgency level: low, medium, or high.
#     **Language:** Identify the language code (e.g., en, de, es).  Refer to language codes for guidance.
#     **Type:** Categorize the nature of the request: Incident, Request, Problem, or Change.

#     Text: {text}

#     Provide the output in JSON format:
#     {{
#     "Department": "...",
#     "Priority": "...",
#     "Language": "...",
#     "Type": "..."
#     }}
#     """
#     response = client.chat.completions.create(
#         model="llama-3.3-70b-versatile",
#         messages=[{"role": "user", "content": prompt}],
#     )
#     classification_result = response.choices[0].message.content.strip()
#     return classification_result

In [5]:
!git clone https://github.com/avaneeshp23/GenAIhackathon.git

Cloning into 'GenAIhackathon'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 42 (delta 15), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 192.47 KiB | 2.47 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [6]:
train_df = pd.read_csv('/content/GenAIhackathon/data/train.csv')

In [32]:
output_df = pd.DataFrame(columns=['ticket_ID', 'department', 'type', 'priority', 'language'])

def classify_ticket_deepseek(ticket_subject, ticket_body, ticket_ID):
    system_prompt = """
    You are an expert IT support ticket classification system. Your task is to analyze the provided ticket information and accurately categorize it based on the following criteria:

    **department:** Assign the ticket to the most appropriate department. Options include: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry.
    **type:** Classify the nature of the request: Incident (report of a disruption or failure), Request (a service or information request), Problem (an underlying issue causing multiple incidents), or Change (a request to modify an existing service or infrastructure).
    **priority:** Determine the urgency level of the ticket: low, medium, or high. Base your decision on the severity and impact of the issue described.
    **language:** Identify the language code (e.g., en, de, es) representing the language used in the ticket. Refer to https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html if needed

    **Important Considerations:**
    **Here are few examples from csv file of train data sample ticket information** ticket_ID	ticket_subject	ticket_body	department	type	priority	language
1001	Discrepancia de facturaciÃ³n en Google Workspace	"Estimado equipo de soporte de TI,

Estoy escribiendo para informar un monto de facturaciÃ³n incorrecto en mi suscripciÃ³n de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.

Saludos,

<name>"	Billing and Payments	Incident	low	es
1002	Urgent Consultation Request for Critical IT Issues	Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest convenience, either via email or phone at <tel_num>. The swift resolution of this situation is imperative to minimize any further disruption to our business. Thank you for your prompt attention to this matter. Kind regards, <name>	Customer Service	Request	high	en
1003	Consulta sobre Servicios de ConsultorÃ­a en TI	"Estimado Servicio de AtenciÃ³n al Cliente,

Mi nombre es <name>, y estoy interesado en obtener mÃ¡s informaciÃ³n sobre sus Servicios de ConsultorÃ­a en TI, centrÃ¡ndome especÃ­ficamente en el desarrollo de software y la administraciÃ³n de servidores. AgradecerÃ­a informaciÃ³n detallada sobre los servicios ofrecidos, precios y proceso de compromiso. Â¿PodrÃ­a proporcionarme esta informaciÃ³n o direccionarme a un representante que pueda ayudarme con mi consulta?

Gracias por su asistencia.

Atentamente,

<name>
<tel_num>
<email_address>"	General Inquiry	Request	medium	es
1004	Demande de mise Ã  jour des dossiers	"Cher service client,

Je vous Ã©cris pour demander une mise Ã  jour de notre systÃ¨me de gestion des dossiers des employÃ©s en accord avec nos opÃ©rations de service de conseil en informatique en cours. Le systÃ¨me actuel nÃ©cessite des amÃ©liorations pour maximiser l'efficacitÃ© et garantir une intÃ©gration harmonieuse avec notre infrastructure informatique existante. Dans le cadre de cette demande, nous souhaiterions discuter de l'Ã©tendue des services et de toute exigence supplÃ©mentaire nÃ©cessaire Ã  la mise en Å“uvre de ces mises Ã  jour.

Veuillez me faire savoir un moment appropriÃ© pour une discussion dÃ©taillÃ©e. Nous vous remercions de votre assistance rapide dans cette affaire.

Merci,

<name>
    """

    user_prompt = f"""
    **Ticket Information:**

    Subject: {ticket_subject}
    Body: {ticket_body}

    **Output Format:**

    Provide your classification in JSON format:

    {{
    "ticket_ID": {ticket_ID},
    "department": "...",
    "type": "...",
    "priority": "...",
    "language": "..."
    }}
    """

    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
    )
    classification_result = response.choices[0].message.content.strip()
    try:
        json_start = classification_result.index("{")
        json_end = classification_result.rindex("}") + 1
        classification_json = json.loads(classification_result[json_start:json_end])
        classification_json['ticket_ID'] = ticket_ID
        print(classification_json)

        return classification_json

    except (json.JSONDecodeError, ValueError) as e:
        print("Failed to parse JSON response:", e)
        return None

In [33]:

output_df = train_df.apply(lambda row: classify_ticket_deepseek(row['ticket_subject'],row['ticket_body'], row['ticket_ID']), axis=1)
#train_df['classification_deepseek'] = train_df.apply(lambda row: rowclassify_ticket_deepseek)
output_df.head()

{'ticket_ID': 1001, 'department': 'Billing and Payments', 'type': 'Incident', 'priority': 'low', 'language': 'es'}
{'ticket_ID': 1002, 'department': 'IT Support', 'type': 'Incident', 'priority': 'high', 'language': 'en'}
{'ticket_ID': 1003, 'department': 'Product Support', 'type': 'Request', 'priority': 'medium', 'language': 'es'}
{'ticket_ID': 1004, 'department': 'IT Support', 'type': 'Change', 'priority': 'medium', 'language': 'fr'}
{'ticket_ID': 1005, 'department': 'Product Support', 'type': 'Incident', 'priority': 'medium', 'language': 'en'}
{'ticket_ID': 1006, 'department': 'Returns and Exchanges', 'type': 'Request', 'priority': 'low', 'language': 'en'}
{'ticket_ID': 1007, 'department': 'Technical Support', 'type': 'Incident', 'priority': 'medium', 'language': 'en'}
{'ticket_ID': 1008, 'department': 'IT Support', 'type': 'Incident', 'priority': 'high', 'language': 'de'}
{'ticket_ID': 1009, 'department': 'IT Support', 'type': 'Incident', 'priority': 'high', 'language': 'es'}


,0
0,"{'ticket_ID': 1001, 'department': 'Billing and..."
1,"{'ticket_ID': 1002, 'department': 'IT Support'..."
2,"{'ticket_ID': 1003, 'department': 'Product Sup..."
3,"{'ticket_ID': 1004, 'department': 'IT Support'..."
4,"{'ticket_ID': 1005, 'department': 'Product Sup..."


In [ ]:
#train_df['classification'] = train_df['text'].apply(classify_ticket)


In [34]:
test_df = pd.read_csv('/content/GenAIhackathon/data/test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...


In [35]:
#test_df['text'] = test_df['ticket_subject'].fillna('') + ' ' + test_df['ticket_body'].fillna('')
test_df.shape

(500, 3)

In [36]:
small_df = test_df[:5]
small_df.head()
small_df['classification'] = small_df.apply(lambda row: classify_ticket_deepseek(row['ticket_subject'],row['ticket_body'], row['ticket_ID']), axis=1)

{'ticket_ID': 1, 'department': 'Technical Support', 'type': 'Incident', 'priority': 'high', 'language': 'it'}
{'ticket_ID': 2, 'department': 'Billing and Payments', 'type': 'Incident', 'priority': 'low', 'language': 'de'}
{'ticket_ID': 3, 'department': 'Product Support', 'type': 'Incident', 'priority': 'low', 'language': 'fr'}
{'ticket_ID': 4, 'department': 'Billing and Payments', 'type': 'Incident', 'priority': 'medium', 'language': 'it'}
{'ticket_ID': 5, 'department': 'Technical Support', 'type': 'Incident', 'priority': 'medium', 'language': 'it'}


<ipython-input-36-458b84d33444>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['classification'] = small_df.apply(lambda row: classify_ticket_deepseek(row['ticket_subject'],row['ticket_body'], row['ticket_ID']), axis=1)


In [37]:
import pandas as pd
import json

# Sample DataFrame with JSON output in 'classification' column

# Function to extract values from JSON and include ticket_ID for mapping
def extract_values_with_id(row):
    if pd.isna(row['classification']):
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': None,
            'type': None,
            'priority': None,
            'language': None
        })

    data = row['classification']
    if isinstance(data, str):
        # Extract the JSON part from the string
        json_str = data.replace("Classification JSON: ", "")
        data = json.loads(json_str.replace("'", "\""))

    if isinstance(data, dict):
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': data.get('department', None),
            'type': data.get('type', None),
            'priority': data.get('priority', None),
            'language': data.get('language', None)
        })
    else:
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': None,
            'type': None,
            'priority': None,
            'language': None
        })


In [38]:
prediction_df = small_df.apply(extract_values_with_id, axis=1)
prediction_df.head()


,ticket_ID,department,type,priority,language
0,1,Technical Support,Incident,high,it
1,2,Billing and Payments,Incident,low,de
2,3,Product Support,Incident,low,fr
3,4,Billing and Payments,Incident,medium,it
4,5,Technical Support,Incident,medium,it


In [39]:
test_df['classification'] = test_df.apply(lambda row: classify_ticket_deepseek(row['ticket_subject'],row['ticket_body'], row['ticket_ID']), axis=1)

{'ticket_ID': 1, 'department': 'Technical Support', 'type': 'Incident', 'priority': 'high', 'language': 'it'}
{'ticket_ID': 2, 'department': 'Billing and Payments', 'type': 'Incident', 'priority': 'low', 'language': 'de'}
{'ticket_id': 3, 'department': 'Product Support', 'type': 'Incident', 'priority': 'medium', 'language': 'fr', 'ticket_ID': 3}
{'ticket_ID': 4, 'department': 'Billing and Payments', 'type': 'Incident', 'priority': 'low', 'language': 'it'}
{'ticket_ID': 5, 'department': 'Technical Support', 'type': 'Incident', 'priority': 'medium', 'language': 'it'}
{'ticket_ID': 6, 'department': 'Product Support', 'type': 'Incident', 'priority': 'medium', 'language': 'it'}
{'ticket_ID': 7, 'department': 'IT Support', 'type': 'Incident', 'priority': 'high', 'language': 'it'}
{'ticket_ID': 8, 'department': 'IT Support', 'type': 'Incident', 'priority': 'high', 'language': 'pt'}
{'ticket_ID': 9, 'department': 'IT Support', 'type': 'Request', 'priority': 'medium', 'language': 'es'}
{'ticket

APITimeoutError: Request timed out.

In [ ]:
test_df.head()

,ticket_ID,ticket_subject,ticket_body,classification
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand...","{'ticket_ID': 1, 'department': 'Product Suppor..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe...","{'ticket_ID': 2, 'department': 'Billing and Pa..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr...","{'ticket_ID': 3, 'department': 'Product Suppor..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques...","{'ticket_ID': 4, 'department': 'Billing and Pa..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...,"{'ticket_ID': 5, 'department': 'Technical Supp..."


In [ ]:
test_df['classification'][0]

{'ticket_ID': 1,
 'department': 'Product Support',
 'type': 'Incident',
 'priority': 'high',
 'language': 'it'}

In [ ]:
# import re
# def extract_json_from_classification(classification_text):
#     try:
#         # Extract JSON part from the classification text
#         json_text = re.search(r'\{.*\}', classification_text, re.DOTALL).group()
#         data = json.loads(json_text)
#         return data
#     except (json.JSONDecodeError, AttributeError) as e:
#         logging.error(f"Error extracting JSON: {e}")
#         return None

# def extract_values_with_id(row):
#     data = extract_json_from_classification(row['classification'])
#     if data:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': data.get('Department', None),
#             'type': data.get('Type', None),
#             'priority': data.get('Priority', None),
#             'language': data.get('Language', None)
#         })
#     else:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': None,
#             'type': None,
#             'priority': None,
#             'language': None
#         })

In [ ]:
predictions_df = test_df.apply(extract_values_with_id, axis=1)

In [ ]:
predictions_df.head()

,ticket_ID,department,type,priority,language
0,1.0,Product Support,Incident,high,it
1,2.0,Billing and Payments,Incident,medium,de
2,3.0,Product Support,Incident,medium,fr
3,4.0,Billing and Payments,Request,medium,it
4,5.0,Technical Support,Incident,high,it


In [ ]:
submission_path = '/content/GenAIhackathon/data/submission.csv'
predictions_df.to_csv(submission_path, index= False)

In [ ]:
submission_df = pd.read_csv(submission_path)
submission_df.head()

,ticket_ID,department,type,priority,language
0,1.0,Product Support,Incident,high,it
1,2.0,Billing and Payments,Incident,medium,de
2,3.0,Product Support,Incident,medium,fr
3,4.0,Billing and Payments,Request,medium,it
4,5.0,Technical Support,Incident,high,it
